In [65]:
#Apriori算法
#构建数据集
def loadDataSet():
    return [[1, 2, 5],[2, 4],[2, 3],[1, 2, 4],[1, 3],[2, 3],[1, 3],[1, 2, 3, 5],[1, 2, 3]]

D=loadDataSet()
print(D)

[[1, 2, 5], [2, 4], [2, 3], [1, 2, 4], [1, 3], [2, 3], [1, 3], [1, 2, 3, 5], [1, 2, 3]]


In [66]:
#扫描loadDataSet返回的事务集，将所有的单项放入C中，构造出单项集
#参数：dataset-List列表，事务集
#返回值：List-冻结后的单项集列表
def createC1(dataSet):
    C = []
    for transaction in dataSet:
        for item in transaction:
            if [ item ] not in C:
                C.append([item])
    C.sort()
#调用frozenset函数将单项集列表冻结,返回冻结单项集列表
    return list(map(frozenset, C))

C1=createC1(D)
print(C1)

[frozenset({1}), frozenset({2}), frozenset({3}), frozenset({4}), frozenset({5})]


In [67]:
#根据minSupport参数设定的最小支持度，计算Ck的项集在原始记录D中的支持度，
#返回满足最小支持度的项集集合，以及所有项集支持度的字典
#参数：D-List列表，事务集
#     Ck-List列表，候选项集
#     minSupport-浮点数，最小支持度阈值
#返回值：retList-List列表，满足最小支持度要求的项集
#       supportData-Dict字典，项集对应的支持度
def scanD(D, Ck, minSupport):
    ssCnt = {}
# 对于每一个候选项集can，检查是否是D的一部分
# 即该候选can是否得到transaction的支持
    for tid in D:
        for can in Ck:     
            if can.issubset(tid):
                ssCnt[can] = ssCnt.get(can, 0) + 1
    numItems = float(len(D))
    retList = []
    supportData = {}
    for key in ssCnt:
#每个项集的支持度
        support = ssCnt[key] / numItems       
#将满足最小支持度的项集，加入retList
        if support >= minSupport:
            retList.insert(0, key)            
#汇总支持度数据
        supportData[key] = round(support,6)
    return retList, supportData

ret1,suD = scanD(loadDataSet(),createC1(loadDataSet()),0.22)
print(ret1)
print(suD)

[frozenset({1}), frozenset({3}), frozenset({2}), frozenset({5}), frozenset({4})]
{frozenset({4}): 0.222222, frozenset({5}): 0.222222, frozenset({2}): 0.777778, frozenset({3}): 0.666667, frozenset({1}): 0.666667}


In [68]:
#由初始候选项集的集合Ck生成新的候选项集，
#参数：Ck-List列表，k-1项集
#      k-整数，要生成的候选项集中的每项包含的元素个数
#返回值：retList-List列表，新生成的候选k项集
def aprioriGen(Ck, k):
    retList = []
    lenCk = len(Ck)
#以下的Ck[i]与Ck[j]均包含k-1个项，如果它们的前k-2项相同，
#则通过或运算将其拼接为一个包含k项的列表元素，加入到retList中
 # for i in range( lenLk ):
    #     for j in range( i + 1, lenLk ):
    #         L1 = list( Lk[ i ] )[ : k - 2 ];
    #         L2 = list( Lk[ j ] )[ : k - 2 ];
    #         L1.sort();L2.sort()
    #         if L1 == L2:
    #             retList.append( Lk[ i ] | Lk[ j ] ) 
    for i in range( lenCk ):
        for j in range( i + 1, lenCk ):
            L1 = Ck[ i ] |Ck[ j ]
            if(len(L1)==k):
                if L1 not in retList:
                    retList.append( L1 ) 
    return retList
    return retList

ret2=aprioriGen(C1, 2)
print(ret2)

[frozenset({1, 2}), frozenset({1, 3}), frozenset({1, 4}), frozenset({1, 5}), frozenset({2, 3}), frozenset({2, 4}), frozenset({2, 5}), frozenset({3, 4}), frozenset({3, 5}), frozenset({4, 5})]


In [69]:
#根据minSupport参数设定的最小支持度，返回所有满足最小支持度的项集，
#参数：D-List列表，事务集
#     minSupport-浮点数，最小支持度阈值
#返回值：L-List列表，所有满足最小支持度要求的项集
#       supportData-Dict字典，项集对应的支持度
def apriori(D, minSupport):
    C1=createC1(D)
    L1, suppData = scanD(D, C1, minSupport)
    L = [L1]
#最初的L1中的每个项集含有一个元素，
#新生成的项集应该含有2个元素，所以 k=2
    k = 2    
    while (len(L[k-2]) > 0):
        Ck = aprioriGen(L[k-2], k)
        Lk, supK = scanD(D, Ck, minSupport)        
#将新的项集的支持度数据加入原来的总支持度字典中
        suppData.update(supK)        
#将符合最小支持度要求的项集加入L
        L.append(Lk)        
#新生成的项集中的元素个数应不断增加
        k += 1
#返回所有满足条件的频繁项集的列表，和所有候选项集的支持度信息
    return L, suppData

L1,suD2=apriori(D,0.22)
print(L1)
print(suD2)

[[frozenset({1}), frozenset({3}), frozenset({2}), frozenset({5}), frozenset({4})], [frozenset({1, 5}), frozenset({1, 2}), frozenset({1, 3}), frozenset({2, 5}), frozenset({2, 3}), frozenset({2, 4})], [frozenset({1, 2, 5}), frozenset({1, 2, 3})], []]
{frozenset({2, 4}): 0.222222, frozenset({5}): 0.222222, frozenset({3}): 0.666667, frozenset({2, 3, 5}): 0.111111, frozenset({1, 2}): 0.444444, frozenset({1, 5}): 0.222222, frozenset({1, 4}): 0.111111, frozenset({3, 5}): 0.111111, frozenset({1, 2, 3, 5}): 0.111111, frozenset({1, 2, 5}): 0.222222, frozenset({1, 2, 4}): 0.111111, frozenset({4}): 0.222222, frozenset({2, 3}): 0.444444, frozenset({2, 5}): 0.222222, frozenset({1}): 0.666667, frozenset({1, 3}): 0.444444, frozenset({1, 3, 5}): 0.111111, frozenset({1, 2, 3}): 0.222222, frozenset({2}): 0.777778}


In [70]:
#计算规则的置信度 
#参数：freqSet-frozenset不可变集合，频繁项集
#     H-不可变集合，频繁项集中所有的元素
#     supportData-dic字典，频繁项集中所有元素的支持度
#     brl-tuple元组，满足可信度条件的关联规则
#     minConf-浮点数，最小置信度阈值，默认值0.7
#返回值：prunedH-List列表，满足最小置信度要求的规则
def calcConf(freqSet, H, supportData, brl, minConf=0.7):
    prunedH = []
    for conseq in H:
        conf = supportData[freqSet] / supportData[freqSet - conseq]
        if conf >= minConf:
            print(freqSet - conseq, '-->', conseq, 'conf:', conf)
            brl.append((freqSet - conseq, conseq, conf))
            prunedH.append(conseq)
    return prunedH

#对频繁项集中元素超过2的项集进行合并
#参数：freqSet-frozenset不可变集合，频繁项集
#     H-不可变集合，频繁项集中所有的元素
#     supportData-dic字典，频繁项集中所有元素的支持度
#     brl-tuple元组，满足可信度条件的关联规则
#     minConf-浮点数，最小置信度阈值，默认值0.7
def rulesFromConseq(freqSet, H, supportData, brl, minConf=0.7):
    m = len(H[0])  
#检查频繁项集是否大到移除大小为 m的子集
    if len(freqSet) > m+1:
        Hmp1 = aprioriGen(H, m+1)
        Hmp1 = calcConf(freqSet, Hmp1, supportData, brl, minConf)
#如果不止一条规则满足要求，进一步递归合并
        if len(Hmp1) > 1:
            rulesFromConseq(freqSet, Hmp1, supportData, brl, minConf)

#根据频繁项集和最小可信度生成规则
#参数：L-list,列表，频繁项集
#     supportData-dict字典，所有项集（不仅仅是频繁项集）的支持度西南西
#     minConf-浮点数，最小置信度阈值，默认值0.7
#返回值：bigRuleList-List列表，满足最小置信度要求的规则
def generateRules(L, supportData, minConf=0.7):
    bigRuleList = []
    for i in range(1, len(L)):
        for freqSet in L[ i ]:
#对于每一个频繁项集的集合freqSet
            H1 = [frozenset([item]) for item in freqSet]
#如果频繁项集中的元素个数大于2，需要进一步合并
            if i > 1:
                rulesFromConseq(freqSet, H1, supportData, bigRuleList, minConf)
            else:
                calcConf(freqSet, H1, supportData, bigRuleList, minConf)
    return bigRuleList

bRlist = generateRules(L1,suD2,0.1)

frozenset({5}) --> frozenset({1}) conf: 1.0
frozenset({1}) --> frozenset({5}) conf: 0.3333328333335833
frozenset({2}) --> frozenset({1}) conf: 0.5714278367349038
frozenset({1}) --> frozenset({2}) conf: 0.6666656666671666
frozenset({3}) --> frozenset({1}) conf: 0.6666656666671666
frozenset({1}) --> frozenset({3}) conf: 0.6666656666671666
frozenset({5}) --> frozenset({2}) conf: 1.0
frozenset({2}) --> frozenset({5}) conf: 0.2857139183674519
frozenset({3}) --> frozenset({2}) conf: 0.6666656666671666
frozenset({2}) --> frozenset({3}) conf: 0.5714278367349038
frozenset({4}) --> frozenset({2}) conf: 1.0
frozenset({2}) --> frozenset({4}) conf: 0.2857139183674519
frozenset({5}) --> frozenset({1, 2}) conf: 1.0
frozenset({2}) --> frozenset({1, 5}) conf: 0.2857139183674519
frozenset({1}) --> frozenset({2, 5}) conf: 0.3333328333335833
frozenset({3}) --> frozenset({1, 2}) conf: 0.3333328333335833
frozenset({2}) --> frozenset({1, 3}) conf: 0.2857139183674519
frozenset({1}) --> frozenset({2, 3}) conf:

In [71]:
if __name__ == '__main__':   
    # 导入数据集
    myDat = loadDataSet()    
# 选择频繁项集
# 选择出支持度不小于2项（共9笔交易，因此支持率为0.22） 的项集作为频繁项集
    L, suppData = apriori(myDat, 0.22)
    print("频繁项集L：", L)
    print("所有候选项集的支持度信息：", suppData)
    print("所有规则的置信度信息：")
    rules = generateRules(L1,suppData,0.1)

频繁项集L： [[frozenset({1}), frozenset({3}), frozenset({2}), frozenset({5}), frozenset({4})], [frozenset({1, 5}), frozenset({1, 2}), frozenset({1, 3}), frozenset({2, 5}), frozenset({2, 3}), frozenset({2, 4})], [frozenset({1, 2, 5}), frozenset({1, 2, 3})], []]
所有候选项集的支持度信息： {frozenset({2, 4}): 0.222222, frozenset({5}): 0.222222, frozenset({3}): 0.666667, frozenset({2, 3, 5}): 0.111111, frozenset({1, 2}): 0.444444, frozenset({1, 5}): 0.222222, frozenset({1, 4}): 0.111111, frozenset({3, 5}): 0.111111, frozenset({1, 2, 3, 5}): 0.111111, frozenset({1, 2, 5}): 0.222222, frozenset({1, 2, 4}): 0.111111, frozenset({4}): 0.222222, frozenset({2, 3}): 0.444444, frozenset({2, 5}): 0.222222, frozenset({1}): 0.666667, frozenset({1, 3}): 0.444444, frozenset({1, 3, 5}): 0.111111, frozenset({1, 2, 3}): 0.222222, frozenset({2}): 0.777778}
所有规则的置信度信息：
frozenset({5}) --> frozenset({1}) conf: 1.0
frozenset({1}) --> frozenset({5}) conf: 0.3333328333335833
frozenset({2}) --> frozenset({1}) conf: 0.571427836734903

In [72]:
import numpy as np
import pandas as pd

#对原始数据进行预处理，原来用0/1表示是否有该银行的信用卡
#本函数将1转换为银行名字缩写
#参数：val-原0/1值，表示没有/拥有该行信用卡
#     colname-读入的dataframe的列名（发卡行缩写）
#返回值：列名（即发卡行缩写），或者0
def val2colindex(val,colname):
    if(val==1):
        return colname
    else:
        return 0

#读取excel文件，将原来二值表格转换为2维列表，用于后续处理
#参数：fname-excel文件名
#返回值：data2Dlist-2维列表
def loadExcel(fname):
    data2Dlist=[]
    pd1 = pd.read_excel(fname,'Sheet1')
    print('读入的数据文件首5行：')
    print(pd1.head())
#将值1转化为列名（即发卡行缩写）
    for cn in pd1.columns:
        if(cn!='ID'):
            pd1[cn]=pd1[cn].apply(lambda val:val2colindex(val,cn))
    print('1转为发卡行名后的数据文件首5行：')
    print(pd1.head())
#将每行转化为一个列表，将ID和0值删去
    for index ,row in pd1.iterrows():
        list1 = row.tolist()
        list1.pop(0)
        while 0 in list1:
            list1.remove(0)
        data2Dlist.append(list1)
#显示2维列表前5项
    print('转化后的2维列表前5项：')
    print(data2Dlist[:4])
#返回该2维列表
    return data2Dlist

myData1=loadExcel('Section4-1 creditcard_info.xlsx')

读入的数据文件首5行：
      ID  gs  ny  zg  js  jt  yc  zs
0  10001   1   0   1   1   0   0   1
1  10002   1   1   1   0   1   1   0
2  10003   0   0   1   0   0   1   1
3  10004   0   0   0   0   1   0   1
4  10005   0   0   0   0   1   1   0
1转为发卡行名后的数据文件首5行：
      ID  gs  ny  zg  js  jt  yc  zs
0  10001  gs   0  zg  js   0   0  zs
1  10002  gs  ny  zg   0  jt  yc   0
2  10003   0   0  zg   0   0  yc  zs
3  10004   0   0   0   0  jt   0  zs
4  10005   0   0   0   0  jt  yc   0
转化后的2维列表前5项：
[['gs', 'zg', 'js', 'zs'], ['gs', 'ny', 'zg', 'jt', 'yc'], ['zg', 'yc', 'zs'], ['jt', 'zs']]


In [73]:
L1,suD2=apriori(myData1,0.22)
print(L1)
print(suD2)

[[frozenset({'ny'}), frozenset({'gs'}), frozenset({'js'}), frozenset({'zg'}), frozenset({'jt'}), frozenset({'zs'}), frozenset({'yc'})], [frozenset({'jt', 'js'}), frozenset({'ny', 'yc'}), frozenset({'zg', 'zs'}), frozenset({'zg', 'gs'}), frozenset({'zs', 'gs'}), frozenset({'jt', 'zs'}), frozenset({'js', 'yc'}), frozenset({'jt', 'ny'}), frozenset({'jt', 'yc'}), frozenset({'jt', 'gs'}), frozenset({'zg', 'yc'}), frozenset({'ny', 'zs'}), frozenset({'jt', 'zg'}), frozenset({'ny', 'zg'}), frozenset({'yc', 'gs'}), frozenset({'js', 'gs'}), frozenset({'ny', 'gs'}), frozenset({'zs', 'yc'})], [frozenset({'jt', 'zs', 'gs'}), frozenset({'jt', 'ny', 'zs'})], []]
{frozenset({'ny', 'zs', 'gs'}): 0.14, frozenset({'ny', 'gs'}): 0.22, frozenset({'js', 'gs'}): 0.22, frozenset({'yc', 'gs'}): 0.26, frozenset({'ny', 'gs', 'yc'}): 0.1, frozenset({'jt', 'zg', 'js'}): 0.12, frozenset({'ny'}): 0.54, frozenset({'ny', 'js'}): 0.2, frozenset({'jt', 'js'}): 0.26, frozenset({'yc'}): 0.48, frozenset({'zs', 'js'}): 0.2,

In [74]:
bRlist = generateRules(L1,suD2,0.6)

frozenset({'zg'}) --> frozenset({'zs'}) conf: 0.6363636363636365
frozenset({'gs'}) --> frozenset({'zs'}) conf: 0.64
frozenset({'zs'}) --> frozenset({'jt'}) conf: 0.7142857142857143
frozenset({'ny'}) --> frozenset({'jt'}) conf: 0.7407407407407407
frozenset({'yc'}) --> frozenset({'jt'}) conf: 0.625
frozenset({'gs'}) --> frozenset({'jt'}) conf: 0.72
frozenset({'zg'}) --> frozenset({'jt'}) conf: 0.6363636363636365
